In [4]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import mofax as mofa
import seaborn as sns
import matplotlib.pyplot as plt
import pyro
from pyro.nn import PyroSample, PyroModule
from pyro.infer import SVI, Trace_ELBO, autoguide
import torch
import torch.nn.functional as F
from torch.nn.functional import softplus
from sklearn.metrics import mean_squared_error
import random
import seaborn as sns
import muon as mu
import anndata

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_device(t): return torch.tensor(t).to(device)

In [2]:
# dir="/scratch/deeplife/"
dir="data/"
pbmc = sc.read_10x_h5(dir+"5k_pbmc_protein_v3_nextgem_filtered_feature_bc_matrix.h5", gex_only=False)
pbmc.var_names_make_unique()
pbmc.layers["counts"] = pbmc.X.copy()

/scratch/venvs/deeplife2/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
protein = pbmc[:, pbmc.var["feature_types"] == "Antibody Capture"].copy()
rna = pbmc[:, pbmc.var["feature_types"] == "Gene Expression"].copy()


In [21]:
class MOFA(PyroModule):
    def __init__(self, Ys: dict[str, torch.Tensor], K, batch_size=128, num_iterations=4000):
        """
        Args:
            Y: Tensor (Samples x Features)
            K: Number of Latent Factors
        """
        super().__init__()
        pyro.clear_param_store()

        self.K = K  # number of factors 

        self.obs_masks = {k: torch.logical_not(torch.isnan(Y)) for k, Y in Ys.items()}
        # a valid value for the NAs has to be defined even though these samples will be ignored later
        self.Ys = {k: torch.nan_to_num(Y, nan=0) for k, Y in Ys.items()}  # data/observations
        
        # assert sample dim same in Ys
        num_samples = set(Y.shape[0] for Y in self.Ys.values())
        assert len(num_samples) == 1
        self.num_samples = next(iter(num_samples))
        self.num_features = {k: v.shape[1] for k, v in self.Ys.items()}
        
        self.batch_size = batch_size
        self.num_iterations = num_iterations
        
        self.latent_factor_plate = pyro.plate("latent factors", self.K) 
        
        
    def model(self):
        """ Creates the model.
        
        The model needs to be created repeatedly (not sure why), in any case, it is important now, when using 
        `subsample_size` batch size to subsample the dataset differently in each train iteration
        """
        
        # needs to be shared, so returns the same indices in one train step
        sample_plate = pyro.plate("sample", self.num_samples, subsample_size=self.batch_size)
        # the plates get assigned a dim, depending on when in the plate hierarchy they are used. Unfortunately we want to use
        #   feature plates once outside and once inside other plate (sample resp. latent_factor plates, see below)
        #   we therefore need to create separate plates for each of those usages
        get_feature_plates = lambda dim: {k: pyro.plate(f"feature_{k}_{dim}", num_feats) for k, num_feats in self.num_features.items()}

        # W matrices for each modality
        Ws = {}
    
        # for each modality create W matrix and alpha vectors
        for m, feature_plate in get_feature_plates(-2).items():
            # the actual dimensions obtained by plates are read from right to left/inner to outer
            with self.latent_factor_plate:
                # Sample alphas (controls narrowness of weight distr for each factor) from a Gamma distribution
                # Gamma parametrization k, theta or eq. a, b; (where k=a and theta=1/b) 
                # (if k integer) Gamma = the sum of k independent exponentially distributed random variables, each of 
                # which has a mean of theta
                alpha = pyro.sample(f"alpha_{m}", pyro.distributions.Gamma(to_device(1.), 1.))
                
                with feature_plate:
                    # sample weight matrix with Normal prior distribution with alpha narrowness
                    Ws[m] = pyro.sample(f"W_{m}", pyro.distributions.Normal(to_device(0.), 1. / alpha))                
                
        # create Z matrix
        # (the actual dimensions are read from right to left/inner to outer)
        with self.latent_factor_plate, sample_plate:
            # sample factor matrix with Normal prior distribution
            Z = pyro.sample("Z", pyro.distributions.Normal(to_device(0.), 1.))
    
        # estimate for Y
        Y_hats = {k: torch.matmul(Z, W.t()) for k, W in Ws.items()}
        
        for m, feature_plate in get_feature_plates(-1).items():
            with feature_plate:
                # sample scale (tau) parameter for each feature-~~sample~~ pair with LogNormal prior (has to be positive)
                # add 0.001 to avoid NaNs when Normal(σ = 0)
                scale_tau = 0.001 + pyro.sample(f"scale_{m}", pyro.distributions.LogNormal(to_device(0.), 1.))
                
                with sample_plate as sub_indices:
                    Y, Y_hat = self.Ys[m][sub_indices, :], Y_hats[m]
                    
                    # masking the NA values such that they are not considered in the distributions
                    obs_mask = self.obs_masks[m][sub_indices, :]                    
                    with pyro.poutine.mask(mask=obs_mask):
                        # # sample scale parameter for each feature-sample pair with LogNormal prior (has to be positive)
                        # scale = pyro.sample("scale", pyro.distributions.LogNormal(0., 1.))
                        
                        # compare sampled estimation to the true observation Y
                        pyro.sample(f"obs_{m}", pyro.distributions.Normal(Y_hat, scale_tau), obs=Y)
                        # pyro.sample("obs", pyro.distributions.NegativeBinomial(Y_hat, scale_tau), obs=Y)

    def train(self):
        # set training parameters
        optimizer = pyro.optim.Adam({"lr": 0.002})
        elbo = Trace_ELBO()
        guide = autoguide.AutoDelta(self.model)
        
        # initialize stochastic variational inference
        svi = SVI(
            model = self.model,
            guide = guide,
            optim = optimizer,
            loss = elbo
        )
        
        train_loss = []
        for j in range(self.num_iterations):
            # calculate the loss and take a gradient step
            # (loss should be already scaled down by the subsample_size)
            loss = svi.step()

            train_loss.append(loss/self.num_samples)
            if j % 200 == 0:
                print("[iteration %04d] loss: %.4f" % (j + 1, loss / self.num_samples))
        
        # Obtain maximum a posteriori estimates for W and Z
        # map_estimates = guide(self.Y)  # not sure why needed Y?
        # "Note that Pyro enforces that model() and guide() have the same call signature, i.e. both callables should take the same arguments."
        with torch.no_grad():
            map_estimates = guide()
        
        return train_loss, map_estimates, guide


In [22]:
mofa = MOFA({
    'rna': torch.tensor(rna.X.toarray(), device=device),
    'protein': torch.tensor(protein.X.toarray(), device=device),
}, K=5, batch_size=128, num_iterations=40000)
loss, map_estimates, trained_guide = mofa.train()


[iteration 0001] loss: 2271781.8510
[iteration 0201] loss: 1406116.4590
[iteration 0401] loss: 918445.2070
[iteration 0601] loss: 625384.0857
[iteration 0801] loss: 335243.3495
[iteration 1001] loss: 300688.9161
[iteration 1201] loss: 271672.2342
[iteration 1401] loss: 163889.6779
[iteration 1601] loss: 107501.3321
[iteration 1801] loss: 93330.2243
[iteration 2001] loss: 75623.1707
[iteration 2201] loss: 55720.1404
[iteration 2401] loss: 21617.3854
[iteration 2601] loss: 17825.7569
[iteration 2801] loss: -1371.1647
[iteration 3001] loss: -23624.2083
[iteration 3201] loss: -27397.4940
[iteration 3401] loss: -27975.6374
[iteration 3601] loss: -34453.2147
[iteration 3801] loss: -40593.5731
[iteration 4001] loss: -50273.8074
[iteration 4201] loss: -54369.6320
[iteration 4401] loss: -55900.9729
[iteration 4601] loss: -56016.3566
[iteration 4801] loss: -65426.0103
[iteration 5001] loss: -66230.6058
[iteration 5201] loss: -70710.3724
[iteration 5401] loss: -68857.9214
[iteration 5601] loss: -

In [60]:
map_estimates

{'alpha_rna': tensor([ 0.6805,  0.7047, 51.2332,  6.3386, 44.6454],
        grad_fn=<ExpandBackward0>),
 'W_rna': tensor([[ 5.7743e-16,  4.8005e-16,  2.3599e-16,  2.2752e-17,  1.7289e-15],
         [ 1.5914e-04,  2.6798e-04,  1.2638e-03,  1.9229e-03,  1.1706e-03],
         [-7.3769e-06, -6.1725e-06,  9.5526e-06, -1.1013e-05,  1.0729e-05],
         ...,
         [-1.4335e-01,  5.1932e-02, -1.5062e-02,  6.9615e-02,  2.6003e-02],
         [ 7.9154e-22, -3.3275e-22, -7.4161e-23, -4.1880e-22,  1.9829e-22],
         [-1.4943e-03, -2.7019e-03, -7.7028e-03, -8.6110e-04,  1.1006e-02]],
        grad_fn=<ExpandBackward0>),
 'alpha_protein': tensor([0.3126, 0.5185, 1.0603, 0.4464, 0.7072], grad_fn=<ExpandBackward0>),
 'W_protein': tensor([[-2.5605, -0.4950, -0.4921, -2.5739,  0.4239],
         [-4.4372, -1.8422,  0.8739, -2.3409,  1.4852],
         [-2.8029, -0.1755, -0.7576, -1.4559,  0.1247],
         [-1.8529, -0.6747,  0.1978, -2.7917,  2.1802],
         [-2.6411, -0.4595, -0.2467, -2.2207,  1

In [36]:
import mofax, h5py, os

os.remove('models/pyro-MOFA.h5')

with h5py.File('models/pyro-MOFA.h5', 'w') as f:
    f['features/rna'] = np.array([*rna.var['feature_types'].index], dtype='S')
    f['features/protein'] = np.array([*protein.var['feature_types'].index], dtype='S')
    f['samples/group1'] = np.array([*pbmc.obs.index], dtype='S')
    f['data/rna/group1'] = rna.X.toarray()
    f['data/protein/group1'] = protein.X.toarray()
    f['expectations/Z/group1'] = map_estimates['Z'].detach().cpu().numpy()
    f['expectations/W/rna'] = map_estimates['W_rna'].detach().cpu().numpy()
    f['expectations/W/protein'] = map_estimates['W_protein'].detach().cpu().numpy()
    f['model_options/likelihoods'] = np.array(['gaussian', 'gaussian'], dtype='S')


mofax_model = mofax.mofa_model('models/pyro-MOFA.h5')